In [1]:


from webdriver_manager.chrome import ChromeDriverManager

driver_path = ChromeDriverManager().install()


def GetBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(driver_path), options=opt)
    except Exception as err:
        print(err)

# def Download(vid, tlang=None):
#     url = f'https://www.youtube.com/watch?v={vid}'
#
#     driver = GetBrowser()
#     driver.get(url)
#
#     while True:
#         try:
#             driver.get(url)
#             break
#         except Exception as err:
#             print(err)
#             time.sleep(2)
#
#     try:
#         cc_element = WebDriverWait(driver, 8).until(
#             EC.presence_of_element_located((By.XPATH, '//button[@class="ytp-subtitles-button ytp-button"]')))
#         cc_element.click()
#     except Exception as err:
#         print(err)
#         raise CCNotAvailable('CC Not Available For This Video')
#
#     perf_logs = driver.get_log('performance')[::-1]
#     for ind, log in enumerate(perf_logs):
#         s = f'\n{str(log)}'
#         links = re.findall(r'https://www.youtube.com/api/timedtext[^"]+', s)
#         if len(links) > 0:
#             while True:
#                 try:
#                     url = links[-1]
#                     response = requests.get(url if tlang is None else f'{url}&tlang={tlang}')
#                     events = response.json()['events']
#
#                     with open('subtitles.json', 'w') as jFile:
#                         json.dump(response.json(), jFile)
#
#                     subtitles_entries = []
#                     sub_index = 1
#
#                     for index, entry in enumerate(events):
#                         segs = entry.get('segs', None)
#                         if segs is not None:
#                             dialogue = ''.join([_.get('utf8', '') for _ in segs]).strip('\n')
#                             if len(dialogue) == 0:
#                                 continue
#                             else:
#                                 tStartMs = entry.get('tStartMs', 0)
#                                 tEndMs = events[index + 1].get('tStartMs', None) if index + 1 < len(
#                                     events) else entry.get('dDurationMs', 0) + tStartMs
#                                 s_entry = f'{sub_index}\n{sub_TimeStamp(tStartMs)} --> {sub_TimeStamp(tEndMs)}\n{dialogue}\n'
#                                 subtitles_entries.append(s_entry)
#                                 sub_index += 1
#
#                     with open('subtitles.srt', 'w', encoding='utf-8') as f:
#                         f.write('\n'.join(subtitles_entries))
#                     break
#                 except Exception as err:
#                     print(err)
#                     time.sleep(1)
#             break
#         elif len(perf_logs) - 1 == ind:
#             raise CCNotAvailable('CC Not Found In Network.Request')


ValueError: There is no such driver by url https://chromedriver.storage.googleapis.com/LATEST_RELEASE_119.0.6045

In [15]:
import threading
import os
import pandas
import time
from tqdm import tqdm
import queue
from lxml import html
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import Chrome, Edge
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



def GetChromeBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    except Exception as err:
        print(err)

def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")
    options.add_argument("--mute-audio")
    options.add_argument("--disable-notifications")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)

resultQ = queue.Queue()
errorQ = queue.Queue()


def Scrapper(driver, url):
    driver.get(url)
    try:
        cc_element = WebDriverWait(driver, 25).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//div[@class="image-grid-image"]')))
    except:
        pass
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * .2);")

    try:
        cc_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="index-showMoreText"]')))
        cc_element.click()
    except:
        pass

    time.sleep(1)

    page_source = driver.page_source
    doc = html.fromstring(page_source)
    d_data = {}

    offers = d_data['offers'] = ''.join(doc.xpath('//*[@class="pdp-offers-price"]/descendant::text()'))
    title = d_data['title'] = ''.join(doc.xpath('//*[@class="pdp-title"]/descendant::text()'))
    name = d_data['name'] = ''.join(doc.xpath('//*[@class="pdp-name"]/descendant::text()'))
    price = d_data['mrp'] = ''.join(doc.xpath('//*[@class="pdp-mrp"]/descendant::text()'))
    pr_container = d_data['pr_container'] = ''.join(
        doc.xpath('//*[@class="pdp-discount-container"]/descendant::text()'))
    price = d_data['price'] = ''.join(doc.xpath('//*[@class="pdp-price"]/descendant::text()'))

    sizes = doc.xpath('//*[@class="size-buttons-tipAndBtnContainer"]')
    sizes_text = []
    for index, _ in enumerate(sizes):
        t = '\n'.join(_.xpath('descendant::text()'))
        sizes_text.append(t)
        d_data[f'size_{index}'] = t

    paras = doc.xpath('//*[@class="pdp-productDescriptorsContainer"]/child::div')
    for __ in paras:
        text = '\n'.join(__.xpath('descendant::text()')).lower()
        if text.__contains__('PRODUCT DETAILS'.lower()):
            d_data['product_details'] = '\n'.join(__.xpath('descendant::text()'))
        if text.__contains__('Size & Fit'.lower()):
            d_data['size_fit'] = '\n'.join(__.xpath('descendant::text()'))
        if text.__contains__('Material & Care'.lower()):
            d_data['material_care'] = '\n'.join(__.xpath('descendant::text()'))
        if text.__contains__('Specifications'.lower()):
            specification = __.xpath('descendant::*[@class="index-row"]')
            for _ in specification:
                d_data[''.join(_.xpath('child::*[1]/text()'))] = ''.join(_.xpath('child::*[2]/text()'))

    images  = '|'.join(doc.xpath('//div[@class="image-grid-image"]/@style'))
    pattern = re.compile(r'https://[^"]+')
    # Find all matches in the input string
    matches = pattern.findall(images)
    d_data['images'] = '/n'.join(matches)
    d_data['product'] = url
    resultQ.put(d_data)


files = os.listdir('myntra_men')

jobs = queue.Queue()


def WorkerLoop(progress_bar: tqdm):
    # driver = GetChromeBrowser(False)
    driver = GetEdgeBrowser(False)
    while not jobs.empty():
        url = jobs.get()
        try:
            Scrapper(driver, url)
        except:
            errorQ.put({'url': url})

            pass
        time.sleep(1)
        progress_bar.update()


i = 'myntra.csv'
path = f'myntra_men/{i}'
if i.__contains__('.csv'):
    data = pandas.read_csv(path)
else:
    data = pandas.read_excel(path)

urls = list(data['product-base href'])
[jobs.put(_) for _ in urls[:5]]
threads = []
pbar = tqdm(total=len(urls))

for worker in range(6):
    thread = threading.Thread(target=WorkerLoop, args=(pbar,))
    thread.start()
    threads.append(thread)

for t in threads:
    t.join()


def qToDf(q):
    l = []
    while not q.empty():
        l.append(q.get())

    df = pandas.DataFrame(l)
    return df


qToDf(resultQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_result.csv', index=False)
qToDf(errorQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_error.csv', index=False)



  1%|          | 5/552 [00:36<1:17:32,  8.50s/it]

In [10]:
qToDf(resultQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_result.csv', index=False)
qToDf(errorQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_error.csv', index=False)

print(f'{"".join(i.split(".")[:-1])}_scrapped_result.csv')

myntra_scrapped_result.csv


In [13]:
import re
input_string = '''
background-image: url("https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/3d5b9dd5-847c-4325-be93-50d7ecdad95a1660045403382VastramayMensGreenSilkBlendJodhpuri1.jpg");
background-image: url("https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/8e1bdde8-5c98-42e1-9c16-443508b2d2811660045403339VastramayMensGreenSilkBlendJodhpuri2.jpg");
background-image: url("https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/4bf62303-4352-468f-bb73-dae7da66fd061660045403396VastramayMensGreenSilkBlendJodhpuri3.jpg");
background-image: url("https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/713b3bc1-ed90-4cdc-b236-4c9b3ecd5d1f1660045403355VastramayMensGreenSilkBlendJodhpuri4.jpg");
background-image: url("https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/a66d88d1-3bfc-4068-b8bd-10603a5a5e751660045403368VastramayMensGreenSilkBlendJodhpuri5.jpg");
'''

# Define the regex pattern
pattern = re.compile(r'https://[^"]+')

# Find all matches in the input string
matches = pattern.findall(input_string)

# Print the matches
for match in matches:
    print(match)

https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/3d5b9dd5-847c-4325-be93-50d7ecdad95a1660045403382VastramayMensGreenSilkBlendJodhpuri1.jpg
https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/8e1bdde8-5c98-42e1-9c16-443508b2d2811660045403339VastramayMensGreenSilkBlendJodhpuri2.jpg
https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/4bf62303-4352-468f-bb73-dae7da66fd061660045403396VastramayMensGreenSilkBlendJodhpuri3.jpg
https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/713b3bc1-ed90-4cdc-b236-4c9b3ecd5d1f1660045403355VastramayMensGreenSilkBlendJodhpuri4.jpg
https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/19451006/2022/8/9/a66d88d1-3bfc-4068-b8bd-10603a5a5e751660045403368VastramayMensGreenSilkBlendJodhpuri5.jpg


In [54]:
import requests
res = requests.get('https://delightshub.com/product-items.php?item_name=a/*%27order*/+/*by*/+1#')
print(res)
print(res.text)

<Response [200]>
b''
